# Erstellung einer Vektordatenbank für das RAG System

Retrieval-Augmented Generation (RAG) ist ein leistungsfähiger Ansatz, der die Stärken von KI-Modellen (z. B. Sprachmodellen) mit externem Wissen aus Wissensbasen oder Vektordatenbanken kombiniert. Das Modell ruft hierbei gezielt relevante Informationen aus der Datenbank ab, um präzisere und kontextuell reichere Antworten zu generieren. Dies ist besonders in komplexen oder faktenbasierten Szenarien hilfreich, da das Modell auf eine große Wissensbasis zugreifen kann, anstatt sich nur auf vortrainiertes Wissen zu verlassen.

Vektordatenbanken sind für RAG unverzichtbar, da sie effiziente Mechanismen bieten, um hochdimensionale Vektoren zu speichern und nach ähnlichen Einträgen zu suchen. Solche Systeme ermöglichen es, Prozesse wie semantische Suche und Wissensabfrage zu automatisieren und zu beschleunigen.

![Vector_database.png](Vector_database.png)

Dieses Notebook zeigt den Aufbau einer Vektordatenbank, um deren Funktionsweise und praktische Anwendung zu demonstrieren. Für die Erstellung der Ausschreibungsunterlagen, bei der lediglich einzelne Dokumente zusammengefasst werden sollen, ist die Implementierung einer Vektordatenbank jedoch nicht zwingend erforderlich, da diese direkt hochgeladen werden können. Dennoch könnte diese Struktur in zukünftigen Projektschritten hilfreich sein, um Prozesse wie die Automatisierung von Abfragen oder das Management großer Datenmengen effizienter zu gestalten.

In [4]:
# Imports
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoFeatureExtractor, AutoModel
import torchvision.transforms as transforms
text_model = SentenceTransformer('all-MiniLM-L6-v2')
image_model_name = "google/vit-base-patch16-224"
feature_extractor = AutoFeatureExtractor.from_pretrained(image_model_name)
image_model = AutoModel.from_pretrained(image_model_name)

# vectordatabase
import chromadb
import base64
from io import BytesIO


c:\Users\sandra.nuissl\Desktop\Automatisierung_von_Ausschreibungen\venv\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# set path
pdf_path = "Data\Produktdatenblatt Migration SE.pdf"

# Example
document_data = process_pdf(pdf_path)

print(document_data.keys())
document_data["context"]

dict_keys(['context', 'images', 'ocr_texts'])


['Migration SEHöhenverstellbare Schreibtische,  Benches und Besprechungstische Die Migration SE Produktfamilie bietet eine Auswahl hochwertiger, flexibler höhenverstellbarer Produkte mit gutem Preis-Leistungs-Verhältnis. Sie können in vielen unterschiedlichen Applikationen eingesetzt werden und tragen zur Steigerung des Wohlbefindens bei. Die schlichte, bewährte Produktreihe gibt den Angestellten die Möglichkeit, selbst zu entscheiden, wann sie im Sitzen bzw. Stehen arbeiten möchten. Ergonomie und LeistungsstärkeWenn die Mitarbeitenden ins Büro kommen, wollen sie nicht nur gemeinsam arbeiten und sich mit Kolleg*innen austauschen, sondern sie erwarten auch leistungsstarke Produkte, die sich auch dadurch auszeichnen, dass man die Höhe selbst einstellen kann. Beim Design der Migration SE-Produktfamilie lag der Fokus auf Flexibilität, Preis/Leistung und dem Nutzer-Wohlbefinden.ProduktlinieSchreibtische + BenchesDie höhenverstellbaren Schreibtische und Benches von Migration SE bieten langfr

## Embedding

Embeddings sind numerische Darstellungen von Daten. In diesem Abschnitt werden die Daten (Texte, Bilder, Texte in Bildern) mithilfe eines vortrainierten Modells oder einer Embedding-Bibliothek in Vektoren umgewandelt.
Embeddings fassen die wesentlichen Merkmale der Daten in einer kompakten, maschinenlesbaren Form zusammen, die für die Speicherung in einer Vektordatenbank geeignet ist.

In [8]:
# Embedding for extraced data
def generate_embeddings(text_model, image_model, feature_extractor, document_data):
    """Generates embeddings for text data and images extracted from a document.

    Args:
        text_model (SentenceTransformer): The model used to encode text data into embeddings.
        image_model (transformers.PreTrainedModel): The model used to encode image data into embeddings.
        feature_extractor (transformers.FeatureExtractor): The feature extractor responsible for preprocessing images for the image model.
        document_data (dict): A dictionary containing the data of the pdf.

    Returns:
        embeddings (list of arrays): Data of the pdf document represented as vektors.
    """
    
    # extract context, OCR text, and image data from document_data
    context_data = document_data["context"]
    ocr_data = document_data["ocr_texts"]
    image_data = document_data["images"]

    # generate text embeddings
    embeddings = {
        "context_embedding": text_model.encode(" ".join(context_data)),
        "ocr_embeddings": [text_model.encode(ocr_text) for ocr_text in ocr_data],
        "image_embeddings": []
    }

    # define image preprocessing to resize the images
    preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    ])

    # generate image embeddings
    for img in image_data:
        # load the image if it's a file path
        if isinstance(img, str):
            img = Image.open(img).convert("RGB")
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # preprocessing transformation
        img_tensor = preprocess(img)
        img_tensor = img_tensor.unsqueeze(0)

        # check if the pixel values are between [0, 1] and normalize if necessary
        if img_tensor.max() <= 1.0 and img_tensor.min() >= 0.0:
            inputs = feature_extractor(images=img_tensor, return_tensors="pt", do_rescale=False)
        else:
            inputs = feature_extractor(images=img_tensor, return_tensors="pt")

        with torch.no_grad():
            # Generate image embedding
            output = image_model(**inputs)

            # Check the dimensions of the last hidden state
            if output.last_hidden_state.dim() == 3:  # Ensure the tensor has 3 dimensions
                img_embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
                embeddings["image_embeddings"].append(img_embedding)
            else:
                raise ValueError(f"Unexpected tensor shape {output.shape}. Expected 3 dimensions.")

    return embeddings

In [ ]:
# Example
embeddings = generate_embeddings(text_model, image_model, feature_extractor, document_data)
print("OCR Embeddings für Bilder:", embeddings["image_embeddings"])

OCR Embeddings für Bilder: [array([[ 3.31620932e-01,  7.25440860e-01,  5.54523528e-01,
        -3.90957475e-01, -5.08184910e-01, -9.89918411e-01,
        -2.42245808e-01, -3.88834387e-01,  8.28510344e-01,
        -4.99536514e-01, -9.60998237e-01,  5.92059553e-01,
         7.79936731e-01, -4.64174956e-01, -6.54313266e-01,
         6.42823339e-01,  7.37555563e-01, -8.60802159e-02,
        -8.28738451e-01, -7.58196831e-01,  2.19619036e-01,
        -2.99000591e-01, -9.31506395e-01, -1.13131888e-01,
         2.25863099e-01, -2.49611549e-02, -1.34117469e-01,
        -6.27186120e-01,  6.65560961e-01,  2.24602476e-01,
        -3.90446723e-01,  3.38999987e-01, -1.02145739e-01,
        -2.96786189e-01,  2.94741392e-01,  6.89983785e-01,
        -1.14835596e+00, -2.64245540e-01, -6.15025043e-01,
        -5.86769342e-01,  1.38303769e+00, -4.91651803e-01,
        -1.16622698e+00, -5.77434421e-01,  1.20170546e+00,
         9.75719392e-01,  1.28773558e+00,  5.34983933e-01,
         2.09351555e-02, -7.

## Vektordatenbank einrichten

In diesem Abschnitt wird die Vektordatenbank initialisiert und die vorbereiteten Embeddings werden eingefügt.
Die Datenbank dient als Speicher und bietet effiziente Such- und Abfragefunktionen, die insbesondere bei großen Datensätzen nützlich sind.

In [10]:
# Function for flatten the embeddings if they are lists of lists
def flatten_list_of_lists(list_of_lists: list) -> list:
    '''Flattens a list of lists into a single list.'''
    return [item for sublist in list_of_lists for item in sublist]

In [11]:
# Function to convert images in an base64 format
def convert_image_to_base64(image: Image) -> str:
    '''Converts an image to a base64 encoded string.'''

    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [12]:
# Function for storing PDF data and embeddings in separate ChromaDB collections
def add_embeddings_to_collections(doc_id, document_data, embeddings, chromadb_client):
    """Stores the PDF data and corresponding embeddings in separate collections within ChromaDB.

    Args:
        doc_id (str): A unique identifier for the document.
        document_data (dict): A dictionary containing details of the document.
        embeddings (dict): A dictionary containing embeddings for the document data.
        chromadb_client: The client for connecting to ChromaDB.
    """

    # extract data
    context_data = document_data["context"]
    ocr_data = document_data["ocr_texts"]
    image_data = document_data["images"]

    # extract embeddings
    context_embedding = embeddings["context_embedding"]
    ocr_embeddings = embeddings["ocr_embeddings"]
    image_embeddings = flatten_list_of_lists(embeddings["image_embeddings"])

    # convert images to Base64 strings if necessary
    image_data_base64 = []
    for img in image_data:
        if isinstance(img, str):
            image_data_base64.append(img)
        else:
            image_data_base64.append(convert_image_to_base64(img))

    # add data and embeddings through the collections
    try:
        # add context data
        context_collection = chromadb_client.get_or_create_collection("context_texts")
        context_collection.add(
            documents=[" ".join(context_data)],
            embeddings=[context_embedding],
            metadatas=[{"doc_id": doc_id, "type": "context"}],
            ids=[f"{doc_id}_context"]
        )

        # add ocr data
        ocr_text_collection = chromadb_client.get_or_create_collection("ocr_texts")
        ocr_text_collection.add(
            documents=ocr_data,
            embeddings=ocr_embeddings,
            metadatas=[{"doc_id": doc_id, "type": "ocr", "ocr_text_num": i} for i in range(len(ocr_data))],
            ids=[f"{doc_id}_ocr_{i}" for i in range(len(ocr_data))]
        )

        # add image data
        image_collection = chromadb_client.get_or_create_collection("image_embeddings")
        image_collection.add(
            documents=image_data_base64,
            embeddings=image_embeddings,
            metadatas=[{"doc_id": doc_id, "type": "image", "image_num": i} for i in range(len(image_data))],
            ids=[f"{doc_id}_image_{i}" for i in range(len(image_data))]
        )

        message = "Embeddings erfolgreich in getrennten Kollektionen gespeichert."
        return True, message

    except Exception as e:
        message = f"Fehler beim Hinzufügen der Embeddings zu den Kollektionen: {e}"
        return False, message


In [ ]:
# Initialize ChromaDB client
client = chromadb.Client()

# id of pdf document
doc_id = pdf_path

# Example
test = add_embeddings_to_collections(doc_id, document_data, embeddings, client)
test

(True, 'Embeddings erfolgreich in getrennten Kollektionen gespeichert.')